In [ ]:
# chatgpt results
from langchain.llms import OpenAI

openai_api_key = ""


In [ ]:
# initializing ChatGPT
llm = OpenAI(
    # model_name="gpt-4-turbo",
    model_name="gpt-4o",
    temperature=0.5,
    max_tokens=200,
    openai_api_key=openai_api_key,
)

In [ ]:
# read in data
# get the text report as input
import pandas as pd
df_text = pd.read_csv('./doc_level_gold_labels.csv')
eg = df_text
input_text = eg.text.to_list()
input_labels = eg.labels.to_list()
eg

In [ ]:
# revised prompt structure
example = """
Symptom List: {"Erythema": ["redness", "redness"], "Pain in extremity": ["sore", "arm", "soreness"], "Pruritus": ["none"]}
Suggest List: {"Erythema": ["redness", "redness"], "Pruritus": ["none"]}
"""
input_template = """
Ignore previous conversations.

Clinical Notes: {text}

First, extract a symptom list including all symptoms from the clinical text above
Then, extract the symptoms that indicating each of the suggested terms below from the symptom list in previous step. 
Include the terms in the output even if the terms are not explicitly mentioned in the provided report, just provide ‘none’ as the result. 
Please follow the order of this list: {suggest} and generate output by following the requirements below:

Requirements:
1. Adverse event means any symptoms or irregular test results. Therefore, procedure description, negative test results, or only the mention of the test itself are not adverse events.
2. If any non symptom, vague mention, or non vaccine related terms appeared in the suggested terms, just provide "none" for them in output. 
3. The output should be include a list of symptoms and in json format like the example below shows.

Example: 
{example}

Here is the output:
"""

In [ ]:
# build up the call
# role:user or system
answer_lst = []
for row in eg.iterrows():
    txt = row[1]['text']
    suggest = row[1]['labels']
    input = input_template.format(text=txt, suggest = suggest, example = example)
    answer = llm(input)
    answer_lst.append(answer)
eg['gpt_result'] = answer_lst

In [ ]:
# eg.final_labels.to_list()

In [ ]:
result_df = eg[['gpt_result', 'final_labels', 'doc_id', 'text']]
f_name = 'gpt_4o_result_temp5_2nd_prompt.json'
result_df.to_json(f_name)


In [ ]:
for r in result_df.iterrows():
    print(r[0])
    print(r[1]['gpt_result'])
    print(r[1]['final_labels'])